#  RAG 성능평가 개요

### **학습 목표:** RAGAS를 사용한 RAG 성능 평가 프로세스를 이해한다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import uuid

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

`(3) langfuse handler 설정`

In [ ]:
from langfuse.callback import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

# 정상 작동 확인
langfuse_handler.auth_check() 

---

## RAG 시스템 성능 평가

- **RAG 기술의 핵심**: 외부 지식 검색과 LLM 결합으로 응답 품질 향상

- **평가 기준**: LLM-as-judge 방식으로 사실성, 관련성, 충실도, 유용성 평가

- **체계적인 A/B 테스트**: 각 컴포넌트별 성능 비교 및 영향도 분석으로 최적 구성 도출

- **평가 방법론**: 오프라인(참조답변 기반), 온라인(실시간), 페어와이즈(비교) 평가 구분

---

### **평가 대상(Evaluation Target)**

- **검색(Retrieval)** 단계: 
    1. 관련 문서와 쿼리 간의 연관성(Relevance)을 통해 검색된 문서가 쿼리의 정보 요구를 얼마나 잘 충족하는지 평가
    1. 관련 문서와 후보 문서 간의 정확성(Accuracy)을 통해 시스템이 적절한 문서를 식별하는 능력을 측정

- **생성(Generation)** 단계:
    1. 응답과 쿼리의 연관성(Relavance)
    1. 응답과 관련 문서 간의 충실도(Faithfulness)
    1. 응답과 샘플 응답 간의 정확성(Correctness)

- 추가 고려사항:
    - **핵심 성능 지표**: Latency(응답 속도), Diversity(검색 다양성), Noise Robustness(잡음 내구성)
    - **안전성 평가**: Negative Rejection(불충분 정보 거부), Counterfactual Robustness(오정보 식별)
    - **사용자 경험**: Readability(가독성), Toxicity(유해성), Perplexity(복잡성) 등 추가 고려


[출처] https://arxiv.org/abs/2405.07437

---

### **평가 데이터셋 구축(Evaluation Dataset)**

- **데이터셋 구성 방식**: LLM 기반 새로운 데이터셋 생성 (Synthetic Data)

- **맞춤형 데이터셋** 구축으로 RAG 시스템의 실용성 평가 강화


- [실습] : **Ragas** (https://docs.ragas.io/en/stable/) 활용

    - **RAG 시스템 평가**를 위한 오픈소스 프레임워크
    - **실용성**: 자동화된 평가 파이프라인 구축 가능
    - **주요 지표**: 

        1. **충실도(Faithfulness)**: 
            - 생성된 답변이 주어진 컨텍스트와 얼마나 일치하는지 평가
        
        2. **답변 관련성(Answer Relevancy)**:
            - 생성된 답변이 주어진 질문과 얼마나 관련이 있는지 평가
        
        3. **컨텍스트 정확도(Context Precision)**:
            - 검색된 컨텍스트들이 얼마나 적절하게 순위가 매겨졌는지 평가
            - 0~1 사이의 값으로, 높을수록 좋음


`(1) LangChain 문서 준비`

In [3]:
from langchain_community.document_loaders import TextLoader

# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.md')) 
korean_data = load_text_files(korean_txt_files)

print('Korean data:')
pprint(korean_data)

Korean data:
[Document(metadata={'source': 'data\\리비안_KR.md'}, page_content='Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**\n\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러\n- **총 자본 (2023):** 91억 4천만 미국 달러\n- **직원 수 (2023년 12월):** 16,790명\n- **웹사이트:** rivian.com\n\n**개요**\n\nRivian은 "스케이트보드" 플랫폼(R1T 및 R1S 모델)을 기반으로 한 전기 스포츠 유틸리티 차량(SUV), 픽업 트럭 및 전기 배달 밴(Rivian EDV)을 생산합니다. R1T 배송은 2021년 말에 시작되었습니다. 회사는 2022년에 미국에서 충전 네트워크를 시작하여 2024년에 다른 차량에도 개방했습니다. 생산 공장은 일리노이 주 노멀에 있으며, 다른 시설은 미국, 캐나다, 영국 및 세르비아의 여러 주에 있습니다.\n\n**역사**\n\n**초창기 (2009–15):**\n\n- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\n- 2011년 Rivian Automotive로 사명 변경.\n- 처음에는 스포츠카 프로토타입(R1)

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=500,
    chunk_overlap=30,
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수:", len(korean_docs))
print("-"*100)
print(korean_docs[0].metadata)
pprint(korean_docs[0].page_content)
print("-"*100)
print(korean_docs[1].metadata)
pprint(korean_docs[1].page_content)

한국어 문서 수: 22
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 '
 '회사입니다.\n'
 '\n'
 '**주요 정보:**\n'
 '\n'
 '- **회사 유형:** 상장\n'
 '- **거래소:** NASDAQ: RIVN\n'
 '- **설립:** 2009년 6월, 플로리다 주 록ledge\n'
 '- **설립자:** R. J. 스캐린지\n'
 '- **본사:** 미국 캘리포니아 주 어바인\n'
 '- **서비스 지역:** 북미\n'
 '- **주요 인물:** R. J. 스캐린지 (CEO)\n'
 '- **제품:** 전기 자동차, 배터리\n'
 '- **생산량 (2023):** 57,232대\n'
 '- **서비스:** 전기 자동차 충전, 자동차 보험\n'
 '- **수익 (2023):** 44억 3천만 미국 달러\n'
 '- **순이익 (2023):** -54억 미국 달러\n'
 '- **총 자산 (2023):** 168억 미국 달러\n'
 '- **총 자본 (2023):** 91억 4천만 미국 달러\n'
 '- **직원 수 (2023년 12월):** 16,790명\n'
 '- **웹사이트:** rivian.com\n'
 '\n'
 '**개요**')
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('**개요**\n'
 '\n'
 'Rivian은 "스케이트보드" 플랫폼(R1T 및 R1S 모델)을 기반으로 한 전기 스포츠 유틸리티 차량(SUV), 픽업 트럭

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=korean_docs,
    embedding=embedding_model,    
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 65


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 로드
vector_store = Chroma(
    embedding_function=embedding_model,    
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 65


`(2) LLM 설정`

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

`(3) Test Data 생성`

In [8]:
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="graduate_researcher",  # 박사과정 연구원: 심도 있는 분석적 질문
        role_description="미국 전기차 시장을 연구하는 한국인 박사과정 연구원으로, 전기차 정책과 시장 동향에 대해 깊이 있는 분석을 하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="masters_student",    # 석사과정 학생: 개념 이해를 위한 질문
        role_description="전기차 산업을 공부하는 한국인 석사과정 학생으로, 미국 전기차 시장의 기초적인 개념과 트렌드를 이해하려 노력하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="industry_analyst",   # 산업 분석가: 실무 중심적 질문
        role_description="한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원으로, 실무적인 시장 데이터와 경쟁사 동향에 관심이 많습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="undergraduate_student",  # 학부생: 기초적인 학습 질문
        role_description="자동차 공학을 전공하는 한국인 학부생으로, 미국 전기차 기술과 시장에 대해 기본적인 지식을 습득하고자 합니다. 한국어만을 사용합니다.",
    )
]

In [9]:
from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(korean_docs, testset_size=20)

Generating Samples: 100%|██████████| 21/21 [00:06<00:00,  3.33it/s]


In [11]:
test_data = dataset.to_pandas()
test_data

TypeError: Cannot convert numpy.ndarray to numpy.ndarray

In [24]:
# CSV 저장
test_data.to_csv('./data/ragas_testset.csv', index=False)

NameError: name 'test_data' is not defined

### **[실습] 평가 데이터셋 합성**

- "data/*_EN.md" 파일들을 문서로 가져와서 청크 길이를 300 토큰 단위로 분할
- ragas 활용 테스트 데이터셋 5개 생성

In [ ]:
from langchain_community.document_loaders import TextLoader

# "data/*_EN.md" 파일 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

english_txt_files = glob(os.path.join('data', '*_EN.md'))
english_data = load_text_files(english_txt_files)

print('English data:')
pprint(english_data)

In [ ]:
# 여기에 코드를 작성하세요.

---

### **평가 지표(Evaluation Metric)**

#### 1) **검색(Retrieval) 평가**  

- **Non-Rank Based Metrics**: Accuracy, Precision, Recall@k 등을 통해 관련성의 이진적 평가를 수행

- **Rank-Based Metrics**: MRR(Mean Reciprocal Rank), MAP(Mean Average Precision)를 통해 검색 결과의 순위를 고려한 평가를 수행

- **RAG 특화 지표**: 기존 검색 평가 방식의 한계를 보완하는 LLM-as-judge 방식 도입

- **포괄적 평가**: 정확도, 관련성, 다양성, 강건성을 통합적으로 측정

#### 2) **생성(Generation) 평가**

- **전통적 평가**: ROUGE(요약), BLEU(번역), BertScore(의미 유사도) 지표 활용

- **LLM 기반 평가**: 응집성, 관련성, 유창성을 종합적으로 판단하는 새로운 접근법 도입 (전통적인 참조 비교가 어려운 상황에서 유용)

- **다차원 평가**: 품질, 일관성, 사실성, 가독성, 사용자 만족도를 포괄적 측정

- **상세 프롬프트**와 **사용자 선호도** 기준으로 생성 텍스트 품질 평가

`(1) RAG 체인 - 평가 대상`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 벡터 저장소 검색기 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# RAG 체인 
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 템플릿 생성
template = """Answer the question based only on the following context:

[Context]
{context}

[Question]
{query}

[Answer]
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = prompt | llm | StrOutputParser()

def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)


query = "Tesla는 언제 누가 만들었나?"

relevant_docs = retriever.invoke(query)
qa_chain.invoke({"context": format_docs(relevant_docs), "query": query})

`(2) 평가 수행을 위한 데이터셋 전처리`

In [ ]:
# 데이터 로드
import pandas as pd
testset = pd.read_excel('data/testset.xlsx')

# 데이터 확인
testset.head()

In [ ]:
from ragas import EvaluationDataset

# 데이터셋 생성
dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in testset.itertuples():
    query = row.user_input   # 사용자 입력
    reference = row.reference  # 참조 답변
    relevant_docs = retriever.invoke(query)  # 검색된 문서
    response = qa_chain.invoke(      # RAG 체인 생성 답변 생성
        {
            "context": format_docs(relevant_docs),
            "query": query,
        })
        #,config={"callbacks": [langfuse_handler]})  
    
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
# 데이터프레임 변환하여 확인 
evaluation_dataset.to_pandas().head()

In [ ]:
# 데이터 저장
evaluation_dataset.to_pandas().to_csv('data/evaluation_dataset.csv', index=False)

### **[실습] RAG 체인 구성 및 추론 실행**

- "data/*_EN.md" 문서에 대한 RAG 체인을 구성
- 이전에 구축한 테스트 데이터셋 5개에 대한 RAG 체인의 출력값을 생성

In [ ]:
# 데이터 로드
testset = pd.read_csv('data/testset_en.csv')
testset.head(2)

In [ ]:
# 여기에 코드를 작성하세요.

`(3) 평가 수행 (상위 모델로 하는 것이 좋음 / Ragas 들어가면 평가 지표가 다양하게 있음)`    

- **LLMContextRecall**
    - 검색 기반 응답 생성 시스템의 **컨텍스트 검색 성능**을 평가 => 검색된 문서와 

- **Faithfulness** 
    - 검색된 컨텍스트에 대한 응답의 **충실도**를 측정하는 메트릭  => hallucination 평가

- **FactualCorrectness**
    - 생성된 응답의 **사실 정확성**을 평가 => hallucination 평가

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# LLM 래퍼 생성
evaluator_llm = LangchainLLMWrapper(llm)

# 평가
result = evaluate(
    dataset=evaluation_dataset,   # 평가 데이터셋
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],   # 평가 메트릭 (하고 싶은 것들 리스트 형태로 넣어주면 됨)
    llm=evaluator_llm,   # LLM 래퍼
    #callbacks=[langfuse_handler],   # 콜백
)

result

In [ ]:
# 결과를 데이터프레임으로 변환 
result.to_pandas()

In [ ]:
# 데이터프레임 저장
result.to_pandas().to_csv('data/evaluation_result.csv', index=False)

### **[실습] RAGAS 평가 수행**

- "data/*_EN.md" 문서에 대한 RAG 생성 답변을 평가 
- RAGAS 평가 메트릭 적용 (LLMContextRecall, Faithfulness, FactualCorrectness)

In [ ]:
# 여기에 코드를 작성하세요.